In [1]:
import pandas as pd
import fasttext
import numpy as np
from gensim.models import FastText

In [2]:
import gensim
gensim.__version__

'3.8.3'

In [2]:
df = pd.read_pickle('../datasets/tweets_cleaned.pkl')

In [150]:
df = df[(df.lang=='es')&(df.favorite_count>4)]

In [151]:
tokenize = lambda text: text.split(' ')

class Corpus():
    def __init__(self, df, size):
        self.df = df
        self.rowids = df.index.values
        self.len = size

    def __iter__(self):
        rowids = np.random.choice(self.rowids, self.len, replace=False)
        for doc_id in rowids:
            yield tokenize(self.df.loc[doc_id].full_text)

    def __len__(self):
        return self.len

In [152]:
docs_s = Corpus(df, 5000)
docs_l = Corpus(df, 50000)

In [157]:
vocab_corpus = df.full_text.apply(tokenize).values

In [161]:
len(vocab_corpus)

1696882

In [190]:
model = FastText.load_fasttext_format("../models/words_fast_text.bin")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [191]:
model.build_vocab(sentences=vocab_corpus, update=True)

In [192]:
model.train(sentences=vocab_corpus, total_examples=len(vocab_corpus), epochs=5, start_alpha=0.0001)  # train

## fasttext
Probar con fasttext a ver si sale mejor

In [3]:
import fasttext
model2 = fasttext.train_unsupervised('../datasets/tweets_corpus.txt', 'skipgram', epoch=20, dim=100, verbose=1)

In [4]:
display(model2.get_nearest_neighbors('cristina'))
print(' ')
display(model2.get_nearest_neighbors('macri'))

[(0.9084580540657043, 'kirchner'),
 (0.8913578391075134, 'cfk'),
 (0.788396954536438, 'kristina'),
 (0.7751791477203369, 'cristino'),
 (0.7686817646026611, 'cristinas'),
 (0.7596039175987244, 'cristinakirchner'),
 (0.7590711116790771, 'cristinafernández'),
 (0.7290238738059998, 'kichner'),
 (0.7122859954833984, 'cristinafernandez'),
 (0.7106063961982727, 'fernández')]

[(0.8794925212860107, 'mauricio'),
 (0.7782546281814575, 'macrismo'),
 (0.7477707862854004, 'macriato'),
 (0.7305310368537903, 'macrivsmacri'),
 (0.7167033553123474, 'macrimafioso'),
 (0.7137525081634521, 'macrista'),
 (0.7105763554573059, 'mauriciomacri'),
 (0.696845531463623, 'macripichetto'),
 (0.6946280002593994, 'michetti'),
 (0.6918284893035889, 'dujovne')]

In [5]:
model2.save_model("../models/words_fast_text.bin")

In [6]:
loaded_model = FastText.load_fasttext_format("../models/words_fast_text.bin")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [7]:
def print_simil(w1,w2):
    print(
        'Similitud entre palabras:',w1,'y',w2, round(loaded_model.wv.similarity(w1, w2), 2)
    )

print_simil('cfk', 'cristina')
print_simil('cfk', 'macri')

Similitud entre palabras: cfk y cristina 0.89
Similitud entre palabras: cfk y macri 0.63


In [37]:
for i in ['👭', '👮‍♂️', '🌈']:
    print('Palabras similares a', i)
    display(loaded_model.wv.most_similar(i,topn=10))
    print('\n')


# print()
# display(loaded_model.wv.most_similar('👮‍♂️',topn=10))
# print()
# display(loaded_model.wv.most_similar('🌈',topn=10))
# print()

Palabras similares a 👭


[('👫', 0.6877906322479248),
 ('👬', 0.6820177435874939),
 ('chicaspochocleras', 0.6516629457473755),
 ('bisexuales', 0.6416600942611694),
 ('🌈', 0.6250035762786865),
 ('🏳', 0.6083692908287048),
 ('35encuentroplurinacional', 0.6034274101257324),
 ('diversidadygénero', 0.5986554622650146),
 ('barbiesimons', 0.5915629267692566),
 ('10añosmatrimonioigualitario', 0.5897146463394165)]



Palabras similares a 👮‍♂️


[('feiman', 0.5960816740989685),
 ('feimann', 0.565120279788971),
 ('opereta', 0.4878638684749603),
 ('elpeor', 0.4789178967475891),
 ('k5n', 0.47482776641845703),
 ('elpeorperiodismodelahistoria', 0.4602394998073578),
 ('sinverguenza', 0.4578441083431244),
 ('tándem', 0.45630669593811035),
 ('operetas', 0.45112377405166626),
 ('mentirosos', 0.448702335357666)]



Palabras similares a 🌈


[('🏳', 0.9444333910942078),
 ('mesdelorgullo', 0.7774953842163086),
 ('10añosdematrimonioigualitario', 0.7723351716995239),
 ('loveislove', 0.7712818384170532),
 ('diadelorgullolgtbi', 0.7707332372665405),
 ('10añosmatrimonioigualitario', 0.7705092430114746),
 ('diadelorgullolgtbiq', 0.7702069878578186),
 ('orgullo2020', 0.7693032622337341),
 ('díadelorgullolgtbi', 0.7672439813613892),
 ('10anosmatrimonioigualitario', 0.761284351348877)]

In [12]:
display(loaded_model.wv.most_similar('cristina',topn=50))
print()
display(loaded_model.wv.most_similar('macri',topn=50))

[('kirchner', 0.9084581136703491),
 ('cfk', 0.8913581371307373),
 ('kristina', 0.788396954536438),
 ('cristino', 0.775179386138916),
 ('cristinas', 0.7686818242073059),
 ('cristinakirchner', 0.7596039772033691),
 ('cristinafernández', 0.7590712308883667),
 ('kichner', 0.7290239334106445),
 ('cristinafernandez', 0.712286114692688),
 ('fernández', 0.7106066346168518),
 ('alberto', 0.703244149684906),
 ('cristinadenuncia', 0.6974077224731445),
 ('cris', 0.6954469680786133),
 ('cristinafkirchner', 0.6922573447227478),
 ('cfkargentina', 0.6913692951202393),
 ('zaninni', 0.6866728067398071),
 ('sagasti', 0.6858562231063843),
 ('cristinista', 0.6853848695755005),
 ('néstor', 0.6832185983657837),
 ('fernandez', 0.6789864301681519),
 ('christina', 0.6720216274261475),
 ('cristina2019', 0.6695528626441956),
 ('berniescristina', 0.6691409349441528),
 ('cristinapresidenta', 0.6659271717071533),
 ('cristinismo', 0.6650446057319641),
 ('albertocristina', 0.6631612777709961),
 ('cristinaenrcv', 0.660

[('mauricio', 0.8794925212860107),
 ('macrismo', 0.7782548069953918),
 ('macriato', 0.7477709054946899),
 ('macrivsmacri', 0.7305310368537903),
 ('macrimafioso', 0.7167034149169922),
 ('macrista', 0.7137525081634521),
 ('mauriciomacri', 0.7105764150619507),
 ('macripichetto', 0.696845531463623),
 ('michetti', 0.6946278810501099),
 ('dujovne', 0.6918286085128784),
 ('vidal', 0.6891368627548218),
 ('mmacri', 0.6822087168693542),
 ('macriísta', 0.6786187887191772),
 ('macrimafia', 0.6769134998321533),
 ('macritemintio', 0.6754363775253296),
 ('carrio', 0.6747381091117859),
 ('scioli', 0.6582700610160828),
 ('gobierno', 0.6567320823669434),
 ('frigerio', 0.656435489654541),
 ('kirchnerismo', 0.6564202308654785),
 ('macris', 0.6553067564964294),
 ('bullrich', 0.6533313393592834),
 ('macricaradepiedra', 0.6526529788970947),
 ('carrió', 0.651379406452179),
 ('caputo', 0.6491774320602417),
 ('garavano', 0.6483981609344482),
 ('pichetto', 0.6480467915534973),
 ('cambiemos', 0.6428484916687012),